In [ ]:
!pip install ultralytics

In [2]:
import yaml
import os
import shutil
from ultralytics import YOLO

In [3]:
### general_utils
def open_yaml_file(yaml_address):
    with open(yaml_address, 'r') as infile:
        yaml_file = yaml.load(infile, Loader=yaml.SafeLoader)
    return yaml_file


def save_yaml_file(yaml_file, save_yaml_address):
    with open(save_yaml_address, 'w') as outfile:
        yaml.dump(yaml_file, outfile, default_flow_style=False)

In [4]:
### train_utils
def train_model(model,
                n_epochs,
                data_yaml_address,
                imgsz=640,
                patience=0,
                device=None,
                batch=16,
                aug_dict={}):
    results = model.train(data=data_yaml_address, epochs=n_epochs, imgsz=imgsz, patience=patience,
                          device=device, batch=batch, **augmentation_dict)
    return results


def load_yolo(model_address):
    model = YOLO(model_address)
    return model

In [5]:
### extra_utils
def change_data_yaml_dataset_locations(yaml_path, new_root_path):
    """
    Using this function to rectify addresses in data.yaml since
    it would contain addresses corresponding to our local directories
    instead of colab-specific addresses.
    """

    yaml_file = open_yaml_file(yaml_path)
    for data_type in ['train', 'test', 'val']:
        yaml_file[data_type] = os.path.join(new_root_path,
                                            data_type)
    save_yaml_file(yaml_file, yaml_path)
    return None

### Specifying dataset location

In [7]:
root_path = "/content/drive/MyDrive/Mhadei Restoration Research/Data for ML Work/Yolo-4"
yaml_path = os.path.join(root_path, "data.yaml")

change_data_yaml_dataset_locations(yaml_path, root_path)

### Specifying augmentation dict

In [8]:
augmentation_dict = {
    "hsv_h": 0.75, # (float) image HSV-Hue augmentation (fraction)
    "hsv_s": 1, # (float) image HSV-Saturation augmentation (fraction)
    "hsv_v": 0.25, # (float) image HSV-Value augmentation (fraction)
    "translate": 0.1, # (float) image translation (+/- fraction)
    "degrees": 10, # (float) image rotation (+/- deg)
    "scale": 0.1, # (float) image scale (+/- gain)
    "shear": 30, # (float) image shear (+/- deg)
    "perspective": 0.0, # (float) image perspective (+/- fraction), range 0-0.001
    "flipud": 0.5, # (float) image flip up-down (probability)
    "fliplr": 0.5, # (float) image flip left-right (probability)
    "bgr": 0.0, # (float) image channel BGR (probability)
    "mosaic": 0.5, # (float) image mosaic (probability)
    "mixup": 0.5, # (float) image mixup (probability)
    "copy_paste": 0.0, # (float) segment copy-paste (probability)
    "auto_augment": "randaugment", # (str) auto augmentation policy for classification (randaugment, autoaugment, augmix)
    "erasing": 0, # (float) probability of random erasing during classification training (0-0.9), 0 means no erasing, must be less than 1.0.
    "crop_fraction": 1.0, # (float) image crop fraction for classification (0.1-1), 1.0 means no crop, must be greater than 0.
}

no_augmentation_dict = {
    "hsv_h": 0, # (float) image HSV-Hue augmentation (fraction)
    "hsv_s": 0, # (float) image HSV-Saturation augmentation (fraction)
    "hsv_v": 0, # (float) image HSV-Value augmentation (fraction)
    "translate": 0, # (float) image translation (+/- fraction)
    "degrees": 0, # (float) image rotation (+/- deg)
    "scale": 0, # (float) image scale (+/- gain)
    "shear": 0, # (float) image shear (+/- deg)
    "perspective": 0.0, # (float) image perspective (+/- fraction), range 0-0.001
    "flipud": 0, # (float) image flip up-down (probability)
    "fliplr": 0, # (float) image flip left-right (probability)
    "bgr": 0.0, # (float) image channel BGR (probability)
    "mosaic": 0, # (float) image mosaic (probability)
    "mixup": 0, # (float) image mixup (probability)
    "copy_paste": 0.0, # (float) segment copy-paste (probability)
    "auto_augment": "randaugment", # (str) auto augmentation policy for classification (randaugment, autoaugment, augmix)
    "erasing": 0, # (float) probability of random erasing during classification training (0-0.9), 0 means no erasing, must be less than 1.0.
    "crop_fraction": 1.0, # (float) image crop fraction for classification (0.1-1), 1.0 means no crop, must be greater than 0.
}

### Training

In [ ]:
train_parameter_dict = {
    "n_epochs": 1000,
    "imgsz": 640,
    "data_yaml_address": yaml_path,
    'patience': 0,
    'device': None,
    'batch': 8
}

yolo_address = "yolov8m-seg.pt"

# load a model
model = load_yolo(yolo_address)

# train a model
results = train_model(model,
                      train_parameter_dict["n_epochs"],
                      train_parameter_dict["data_yaml_address"],
                      imgsz=train_parameter_dict["imgsz"],
                      patience=train_parameter_dict["patience"],
                      device=train_parameter_dict['device'],
                      batch=train_parameter_dict['batch'],
                      aug_dict=augmentation_dict)

print("Results have been saved in:", results.save_dir)

In [11]:
# To store final results inside drive
shutil.move(str(results.save_dir), 'drive/MyDrive/Mhadei Restoration Research/Modelling Results/')

'drive/MyDrive/Mhadei Restoration Research/Modelling Results/train2'

### Rough

These functions should have worked but they don't work. Very weird. Haven't played around with them enough or tested them enough though.

In [ ]:
#### changing augmentation hyperparameters
def change_config_yaml(default_yaml_address, param_dict, new_address):
    create_new_config_yaml(default_yaml_address, param_dict, new_address)
    change_default_yaml_config_address(default_yaml_address, new_address)
    return None


def create_new_config_yaml(source_yaml_address,
                         param_dict,
                         new_yaml_save_address):
    source_yaml = open_yaml_file(source_yaml_address)
    for param, val in param_dict.items():
        try:
            source_yaml[param] = val
        except:
            print(f"Problem with parameter {param}")

    save_yaml_file(source_yaml, new_yaml_save_address)
    return None


def change_default_yaml_config_address(default_yaml_address, new_address):
    default_yaml = open_yaml_file(default_yaml_address)
    default_yaml['cfg'] = new_address
    save_yaml_file(default_yaml, default_yaml_address)
    return None


def change_default_yaml_with_augmentation_params(config_yaml_address,
                                                 augmentation_dict):

    cfg_yaml = open_yaml_file(config_yaml_address)
    create_true_default_config_yaml(cfg_yaml)

    for augmentation_parameter, augmentation_value in augmentation_dict.items():
        try:
            cfg_yaml[augmentation_parameter] = augmentation_value
        except:
            print(f"Problem with augementation parameter {augmentation_parameter}")

    save_yaml_file(cfg_yaml, config_yaml_address)
    return None


def create_true_default_config_yaml(config_yaml):
    main_dir_files = os.listdir("/content/")
    if 'true_default.yaml' in main_dir_files:
        return None
    save_yaml_file(config_yaml, '/content/true_default.yaml')
    return None